### 예제
1. csv 폴더에 있는 파일 4개 로드 (tran_1, tran_2, tran_d_1, tran_d_2)
2. tran_1, tran_2 단순하게 행을 추가 결합 (tran)
3. tran_d_1, tran_d_2 단순하게 행을 추가 결합 (tran_d)
4. 2번, 3번 과정에서 결합한 데이터프레임을 특정 조건에 맞춰서 left(tran)를 기준으로 열 결합(transaction)

In [31]:
import pandas as pd

In [32]:
## 파일 로드 
tran_1 = pd.read_csv("../csv/tran_1.csv")
tran_2 = pd.read_csv("../csv/tran_2.csv")
tran_d_1 = pd.read_csv("../csv/tran_d_1.csv")
tran_d_2 = pd.read_csv("../csv/tran_d_2.csv")

In [33]:
## 단순하게 행 결합
tran = pd.concat([tran_1, tran_2], axis=0, ignore_index=True)

In [34]:
tran_d = pd.concat([tran_d_1, tran_d_2], axis = 0, ignore_index=True)

In [35]:
tran.head(1)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502


In [36]:
tran_d.head(1)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1


In [37]:
## 특정 조건에 맞춰서 tran을 기준으로 열을 추가 결합
## 특정 조건(transaction_id)
## how (left = tran)
transaction = pd.merge(
    left = tran, 
    right=tran_d, 
    on='transaction_id', 
    how = 'left'
)

### 데이터 결합2
1. 파일 로드 (customer_master, item_master)
2. 1번 과정에서 불러온 2개의 데이터프레임을 transaction에 조건에 맞게 조인 결합
3. 결합이 된 데이터프레임에서 quantity컬럼과 item_price 컬럼을 가지고 total_price 새로운 컬럼을 추가(quantity * item_price)
4. payment_date 컬럼을 시계열 데이터로 변경

In [38]:
## 파일 로드 
customer_master = pd.read_csv("../csv/customer_master.csv")
item_master = pd.read_csv("../csv/item_master.csv")

In [39]:
transaction.head(1)

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1


In [40]:
customer_master.head(1)

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0


In [41]:
## transaction, customer_master 의 특정 조건으로 결합
## 조건 -> customer_id
join_data = pd.merge(
    transaction, 
    customer_master, 
    on = 'customer_id', 
    how = 'left'
)

In [42]:
join_data.head(1)

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1,XX,C01,M,2018-07-01 00:00:00,NaN,CA1,0.0


In [43]:
item_master.head(1)

,item_id,item_name,item_price
0,S001,PC-A,50000


In [44]:
total_df = pd.merge(
    join_data, 
    item_master, 
    on = 'item_id', 
    how = 'left'
)

In [45]:
total_df.head(1)

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity,name,class,gender,start_date,end_date,campaign_id,is_deleted,item_name,item_price
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1,XX,C01,M,2018-07-01 00:00:00,NaN,CA1,0.0,PC-E,210000


In [47]:
total_df['total_price'] = total_df['quantity'] * total_df['item_price']

In [48]:
### payment_date 컬럼의 데이터 타입을 시계열 변경
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7144 entries, 0 to 7143
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  7144 non-null   object 
 1   price           7144 non-null   int64  
 2   payment_date    7144 non-null   object 
 3   customer_id     7144 non-null   object 
 4   detail_id       7144 non-null   int64  
 5   item_id         7144 non-null   object 
 6   quantity        7144 non-null   int64  
 7   name            6069 non-null   object 
 8   class           6069 non-null   object 
 9   gender          6069 non-null   object 
 10  start_date      6069 non-null   object 
 11  end_date        1978 non-null   object 
 12  campaign_id     6069 non-null   object 
 13  is_deleted      6069 non-null   float64
 14  item_name       7144 non-null   object 
 15  item_price      7144 non-null   int64  
 16  total_price     7144 non-null   int64  
dtypes: float64(1), int64(5), object(1

In [49]:
total_df.head(1)

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity,name,class,gender,start_date,end_date,campaign_id,is_deleted,item_name,item_price,total_price
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1,XX,C01,M,2018-07-01 00:00:00,NaN,CA1,0.0,PC-E,210000,210000


In [54]:
total_df['payment_date'] = pd.to_datetime(
    total_df['payment_date'], 
    format = '%Y-%m-%d %H:%M:%S')

In [55]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7144 entries, 0 to 7143
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   transaction_id  7144 non-null   object        
 1   price           7144 non-null   int64         
 2   payment_date    7144 non-null   datetime64[ns]
 3   customer_id     7144 non-null   object        
 4   detail_id       7144 non-null   int64         
 5   item_id         7144 non-null   object        
 6   quantity        7144 non-null   int64         
 7   name            6069 non-null   object        
 8   class           6069 non-null   object        
 9   gender          6069 non-null   object        
 10  start_date      6069 non-null   object        
 11  end_date        1978 non-null   object        
 12  campaign_id     6069 non-null   object        
 13  is_deleted      6069 non-null   float64       
 14  item_name       7144 non-null   object        
 15  item

In [63]:
## 시계열 데이터에서 년도-월 부분만 따로 추출 -> payment_month 컬럼 생성
total_df['payment_month'] = total_df['payment_date'].dt.strftime("%Y-%m")

## 그룹화 예제
1. paymeny_month를 기준으로 그룹화
2. total_price의 합계, price의 합계를 출력
3. 2개의 합계를 matplotlib을 이용하여 라인그래프로 시각화 (subplot 이용)